# Solution

In [1]:
%matplotlib inline

## Preparation

In [2]:
import pandas as pd

io_params = {
    'parse_dates': ['epoch']
}

train = pd.read_csv('data/train.csv', **io_params)

Remove the duplicates.

In [3]:
import datetime as dt
import tqdm

dtypes = train.dtypes.to_dict()

cols_to_shift = train.columns.difference(['x_sim', 'y_sim', 'z_sim', 'Vx_sim', 'Vy_sim', 'Vz_sim'])

train_sats = []

for sat_id in tqdm.tqdm(train['sat_id'].unique(), position=0):
    
    g = train.query('sat_id == @sat_id').copy()
    dups = g[g['epoch'].diff() < dt.timedelta(seconds=60)].index
    
    for i in reversed(dups):
        g.loc[i:, cols_to_shift] = g.loc[i:, cols_to_shift].shift(-1)
        
    g = g.drop(g[g['x'].isnull()].index)
    g['percent'] = pd.np.arange(1, len(g) + 1) / len(g)
        
    train_sats.append(g)
    
train = pd.concat(train_sats).astype(dtypes)

100%|██████████| 600/600 [00:05<00:00, 101.84it/s]


Merge train and test.

In [4]:
test = pd.read_csv('data/Track 1/test.csv', **io_params)
data = pd.concat((train, test), sort=False)
data['is_train'] = data['x'].notnull()
data = data.sort_values(['sat_id', 'epoch'])
data['is_track_1'] = data['sat_id'].isin(data.query('not is_train')['sat_id'].unique())

Implement SMAPE.

In [5]:
import numpy as np

def smape(y_true, y_pred): 
    return np.mean(np.abs((y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))))

Implement a generic auto-regressive model.

In [6]:
class ARModel:
    
    def __init__(self, p, model):
        self.p = p
        self.model = model
    
    def fit(self, path):
        
        n = path.strides[0]
        X = np.lib.stride_tricks.as_strided(path, shape=(path.shape[0], self.p), strides=(n, n))[:-self.p]
        Y = path[self.p:]
        
        # Save the most recent history for later usage
        # Conceptually history is a list, but we give it an extra dimension because sklearn eats matrices
        self.history = path[-self.p:].reshape(1, -1)
        
        self.model.fit(X, Y)
        
    def forecast(self, steps):
        
        history = self.history.copy()
        predictions = np.empty(steps)
        
        for i in range(steps):
            
            y_pred = self.model.predict(history)[0]    
            predictions[i] = y_pred
            
            # Shift forward (faster than np.roll)
            history[0, :-1] = history[0, 1:]
            history[0, -1] = y_pred
            
        return predictions

## Local validation

In [7]:
import sklearn
from sklearn import compose
from sklearn import linear_model
from sklearn import pipeline
from sklearn import preprocessing
import tqdm

preds = []


class Pipeline:
    """Barebones implementation with less overhead than sklearn."""
    
    def __init__(self, *steps):
        self.steps = steps
    
    def fit(self, X, y):
        for transformer in self.steps[:-1]:
            X = transformer.fit_transform(X, y)
        self.steps[-1].fit(X, y)
        return self
    
    def predict(self, X):
        for transformer in self.steps[:-1]:
            X = transformer.transform(X)
        return self.steps[-1].predict(X)


class StandardScaler(preprocessing.StandardScaler):
    """Barebones implementation with less overhead than sklearn."""
    
    def transform(self, X):
        return (X - self.mean_) / self.var_ ** .5
    
    
class LinearRegression(linear_model.LinearRegression):
    """Barebones implementation with less overhead than sklearn."""
    
    def predict(self, X):
        return np.dot(X, self.coef_) + self.intercept_

    
model = ARModel(
    p=48,
    model=Pipeline(
        StandardScaler(),
        LinearRegression()
    )
)

train = data.query('is_train')

for sat, g in tqdm.tqdm(train.assign(is_fit=train.eval('percent < .6')).groupby('sat_id'), position=0):
    
    fit = g.query('is_fit')
    val = g.query('not is_fit')
    
    for var in ('x', 'y', 'z', 'Vx', 'Vy', 'Vz'):
        
        model.fit(fit[var].to_numpy())
        pred = model.forecast(len(val))

        preds.append(pd.DataFrame({
            'sat_id': sat,
            'epoch': val['epoch'],
            'y_true': val[var],
            'y_pred': pred,
            'variable': var
        }))
        
preds = pd.concat(preds)

100%|██████████| 600/600 [00:36<00:00, 16.61it/s]


In [8]:
smapes = preds.groupby(['sat_id', 'variable']).apply(lambda g: smape(g['y_true'], g['y_pred']))
mean_smape = smapes.mean()
100 * (1 - mean_smape)

97.5713206421313

Save the validation SMAPEs for further comparison and blending with other methods.

In [9]:
smapes.rename('smape').to_csv('results/ar_val_scores.csv', header=True)
!head results/ar_val_scores.csv

sat_id,variable,smape
0,Vx,0.0005970701556173192
0,Vy,0.0003636738878867748
0,Vz,0.00017282723316789216
0,x,0.0017613854970023823
0,y,0.00025717131530660314
0,z,0.0006687015293489595
1,Vx,0.0031092783188818502
1,Vy,0.0009305197943052433
1,Vz,0.0037287808879610447


Use estimated positions to predict speed

In [10]:
truth_val = preds.groupby('sat_id').apply(lambda g: g.pivot_table(index=['epoch'], columns='variable', values='y_true')).reset_index()
preds = preds.groupby('sat_id').apply(lambda g: g.pivot_table(index=['epoch'], columns='variable', values='y_pred')).reset_index()
preds.head()

variable,sat_id,epoch,Vx,Vy,Vz,x,y,z
0,0,2014-01-19 14:08:39.284999936,-1.526807,0.052730,-3.521541,3367.336471,29830.399180,6888.635609
1,0,2014-01-19 14:55:22.286000128,-1.574304,-1.215272,-3.593862,-1020.504298,28255.630705,-3181.218266
2,0,2014-01-19 15:42:05.286000128,-1.400334,-2.601127,-3.152064,-5249.933015,22900.068540,-12774.903885
3,0,2014-01-19 16:28:48.287000064,-0.966748,-3.744041,-2.118757,-8623.524684,13899.066184,-20288.238119
4,0,2014-01-19 17:15:31.287000064,-0.366184,-4.263117,-0.718761,-10510.274380,2505.095957,-24302.995039


In [11]:
speed_preds = preds.copy()

#Get the time difference between two observations in seconds
preds["time_delta"] = preds.groupby('sat_id').epoch.diff().bfill()
preds["time_delta"] = preds['time_delta'].apply(lambda t: t.total_seconds())

for var in ('x', 'y', 'z'):
    speed_preds[f"V{var}"] = preds.groupby('sat_id')[var].diff().shift(-1).ffill() / preds.time_delta

speed_preds.head()

variable,sat_id,epoch,Vx,Vy,Vz,x,y,z
0,0,2014-01-19 14:08:39.284999936,-1.565408,-0.561815,-3.592526,3367.336471,29830.399180,6888.635609
1,0,2014-01-19 14:55:22.286000128,-1.508893,-1.910653,-3.422648,-1020.504298,28255.630705,-3181.218266
2,0,2014-01-19 15:42:05.286000128,-1.203565,-3.211203,-2.680462,-5249.933015,22900.068540,-12774.903885
3,0,2014-01-19 16:28:48.287000064,-0.673118,-4.064918,-1.432307,-8623.524684,13899.066184,-20288.238119
4,0,2014-01-19 17:15:31.287000064,-0.063926,-4.224444,-0.023003,-10510.274380,2505.095957,-24302.995039


In [12]:
list_smape = []
for sat, g in speed_preds.groupby('sat_id'):
    truth = truth_val.query('sat_id == @sat')
    for var in ('Vx', 'Vy', 'Vz'):
        list_smape.append(pd.DataFrame({
            'sat_id': [sat],
            'variable': [var],
            'smape': [smape(truth[var].values, g[var].values)]
        }))
list_smape = pd.concat(list_smape)
list_smape.head()

,sat_id,variable,smape
0,0,Vx,0.140543
0,0,Vy,0.164085
0,0,Vz,0.155479
0,1,Vx,0.153289
0,1,Vy,0.149691


In [13]:
list_smape.to_csv('results/ar_speed_val_scores.csv', header=True, index=False)
!head results/ar_speed_val_scores.csv

sat_id,variable,smape
0,Vx,0.1405427885947127
0,Vy,0.16408502260307983
0,Vz,0.15547919421757386
1,Vx,0.15328904162060672
1,Vy,0.14969053004583935
1,Vz,0.1665692143367779
2,Vx,0.1543306255096227
2,Vy,0.15838883756807873
2,Vz,0.16928327067205654


## Track 1

Make predictions for the test set.

In [14]:
preds = []

train_sats = data.query('is_train and is_track_1')
test_sats = data.query('not is_train and is_track_1')

for sat in tqdm.tqdm(test_sats['sat_id'].unique(), position=0):

    train = train_sats.query('sat_id == @sat')
    test = test_sats.query('sat_id == @sat')

    for var in ('x', 'y', 'z', 'Vx', 'Vy', 'Vz'):

        model.fit(train[var].to_numpy())
        pred = model.forecast(len(test))

        preds.append(pd.DataFrame({
            'sat_id': test['sat_id'],
            'id': test['id'],
            'epoch': test['epoch'],
            'y_pred': pred,
            'variable': var
        }))
        
preds = pd.concat(preds)
preds.head()

100%|██████████| 300/300 [00:31<00:00,  9.46it/s]


,sat_id,id,epoch,y_pred,variable
0,1,3927,2014-02-01 00:01:45.162,-24791.216496,x
1,1,3928,2014-02-01 00:22:57.007,-21087.026330,x
2,1,3929,2014-02-01 00:44:08.852,-16579.831302,x
3,1,3930,2014-02-01 01:05:20.697,-11202.732371,x
4,1,3931,2014-02-01 01:26:32.542,-4934.229045,x


In [15]:
len(preds)

1704426

The predictions are melted, so we unmelt them.

In [16]:
preds = preds.groupby('sat_id').apply(lambda g: g.pivot_table(index=['id', 'epoch'], columns='variable', values='y_pred')).reset_index()
preds.head()

variable,sat_id,id,epoch,Vx,Vy,Vz,x,y,z
0,1,3927,2014-02-01 00:01:45.162,2.614250,-1.303894,1.087091,-24791.216496,-10910.678758,6570.591143
1,1,3928,2014-02-01 00:22:57.007,3.219142,-0.994539,0.895780,-21087.026330,-12384.401306,7840.063865
2,1,3929,2014-02-01 00:44:08.852,3.877770,-0.539693,0.601826,-16579.831302,-13379.440557,8806.129427
3,1,3930,2014-02-01 01:05:20.697,4.582811,0.157078,0.134822,-11202.732371,-13655.956823,9298.513352
4,1,3931,2014-02-01 01:26:32.542,5.251436,1.293476,-0.653268,-4934.229045,-12796.448989,9015.294227


Take into account the shifts.

In [17]:
correct_preds = []

cols_to_shift = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

for _, g in tqdm.tqdm(preds.groupby('sat_id'), position=0):
    
    g = g.copy()
    dups = g[g['epoch'].diff() < dt.timedelta(seconds=60)].index
    
    for i in dups:
        g.loc[i:, cols_to_shift] = g.loc[i:, cols_to_shift].shift()
    g[cols_to_shift] = g[cols_to_shift].ffill()
    
    correct_preds.append(g)
    
correct_preds = pd.concat(correct_preds)

100%|██████████| 300/300 [00:01<00:00, 234.02it/s]


In [18]:
correct_preds.head()

variable,sat_id,id,epoch,Vx,Vy,Vz,x,y,z
0,1,3927,2014-02-01 00:01:45.162,2.614250,-1.303894,1.087091,-24791.216496,-10910.678758,6570.591143
1,1,3928,2014-02-01 00:22:57.007,3.219142,-0.994539,0.895780,-21087.026330,-12384.401306,7840.063865
2,1,3929,2014-02-01 00:44:08.852,3.877770,-0.539693,0.601826,-16579.831302,-13379.440557,8806.129427
3,1,3930,2014-02-01 01:05:20.697,4.582811,0.157078,0.134822,-11202.732371,-13655.956823,9298.513352
4,1,3931,2014-02-01 01:26:32.542,5.251436,1.293476,-0.653268,-4934.229045,-12796.448989,9015.294227


In [19]:
correct_preds.duplicated(['sat_id', 'id']).sum()

0

Save the predictions for track 1. 

In [20]:
correct_preds[['id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']].to_csv('results/ar_track_1.csv', index=False)
!head -5 results/ar_track_1.csv

id,x,y,z,Vx,Vy,Vz
3927,-24791.216496424422,-10910.678757601636,6570.591142685116,2.614250091109964,-1.3038936610685237,1.0870908640772394
3928,-21087.026330143886,-12384.40130584989,7840.063864809623,3.2191424954744186,-0.9945388414207204,0.8957800688453397
3929,-16579.831302143713,-13379.440557068332,8806.129427121372,3.8777700588719815,-0.539693277312599,0.601826350898311
3930,-11202.732370859067,-13655.956823114819,9298.513352163589,4.582811063189779,0.15707843779166794,0.13482178502177014


In [21]:
speed_preds = correct_preds.copy()

#Get the time difference between two observations in seconds
correct_preds["time_delta"] = correct_preds.groupby('sat_id').epoch.diff().bfill()
correct_preds["time_delta"] = correct_preds['time_delta'].apply(lambda t: t.total_seconds())

for var in ('x', 'y', 'z'):
    speed_preds[f"V{var}"] = correct_preds.groupby('sat_id')[var].diff().shift(-1).ffill() / correct_preds.time_delta

speed_preds.head()

variable,sat_id,id,epoch,Vx,Vy,Vz,x,y,z
0,1,3927,2014-02-01 00:01:45.162,2.912454,-1.158728,0.998135,-24791.216496,-10910.678758,6570.591143
1,1,3928,2014-02-01 00:22:57.007,3.543824,-0.782359,0.759578,-21087.026330,-12384.401306,7840.063865
2,1,3929,2014-02-01 00:44:08.852,4.227794,-0.217413,0.387141,-16579.831302,-13379.440557,8806.129427
3,1,3930,2014-02-01 01:05:20.697,4.928669,0.675796,-0.222684,-11202.732371,-13655.956823,9298.513352
4,1,3931,2014-02-01 01:26:32.542,5.426464,2.186445,-1.291792,-4934.229045,-12796.448989,9015.294227


In [22]:
speed_preds[['id', 'x', 'y', 'z', 'Vx', 'Vy', 'Vz']].to_csv('results/ar_speed_track_1.csv', index=False)
!head -5 results/ar_speed_track_1.csv

id,x,y,z,Vx,Vy,Vz
3927,-24791.216496424422,-10910.678757601636,6570.591142685116,2.9124540854274974,-1.1587281062144,0.9981347743824968
3928,-21087.026330143886,-12384.40130584989,7840.063864809623,3.5438241515280344,-0.7823588968926579,0.7595780636097552
3929,-16579.831302143713,-13379.440557068332,8806.129427121372,4.227794213355123,-0.21741349460546422,0.3871414559495984
3930,-11202.732370859067,-13655.956823114819,9298.513352163589,4.928669237259334,0.6757960556273876,-0.22268368038897257


## Track 2

In [23]:
import copy

models = {}

train_sats = data.query('is_train and not is_track_1')  # is_track_2 = not is_track_1 

for sat, g in tqdm.tqdm(train_sats.groupby('sat_id'), position=0):
    
    models[sat] = {}

    for col in ('x', 'y', 'z', 'Vx', 'Vy', 'Vz'):

        path = g[col].to_numpy()
        model.fit(path)
        models[sat][col] = copy.deepcopy(model)

100%|██████████| 300/300 [00:07<00:00, 42.75it/s]


Save the models and the histories.

In [24]:
import joblib

joblib.dump(models, 'track_2/ar_models.pkl')
!du -h track_2/ar_models.pkl

4,8M	track_2/ar_models.pkl
